# PROJECT OVERVIEW
![overview](imgs/project_overview.jpg)

# Space
Loads and stores meshes and the functional space for running Finite Element Methods (FEM). It includes useful mesh-related functions, such as conversions from local to global vectors and obtaining local tangent bases.

**Note:** Agents should use only geometry-central's mesh. MFEM's mesh is reserved for solving PDEs.

# Agent
Good and evil agents share functionalities (they belong to the same class). They differ in physical properties (radius and speed) and the `agent_type` attribute (+1 for good cells, -1 for evil cells).

Main functionalities:
- `move(...)`: Moves the agent along geodesics based on previously stored velocity (speed and direction are separated).

  **Note:** Implement agent boundary conditions here. Reflective boundary conditions are default for planar meshes; remove/comment this code for closed surfaces.

- `findFacesWithinRadius(double coverage_radius)`: Finds vertices within `coverage_radius` from the agent (`gc_position`), then returns adjacent faces. If no argument is provided, the agent’s radius is used. Results with the agent’s radius are cached to avoid recomputation when the agent does not move.

  **Note:** Euclidean distance is default for planar meshes. For complex geometries, cumulative edge-length may provide better estimates. Note this approximation tends to overestimate covered surfaces.

- Ligand-receptor dynamics implementing realistic BPRW (see [link](https://doi.org/10.3389/fmicb.2015.00503)).

  **Note:** The implemented ODE is slightly different:
  $$\begin{cases}
  R_m' &= k_r R_m^* - \int_{G_m} Q_m \\
  LR_m' &=  \int_{G_m} Q_m   - k_i LR_m \\
  {R^{*}_m}' &= k_i LR_m - k_r R^*_m
  \end{cases}$$
  where $G_m$ is the area covered by the m-th good agent.

# Collision-Manager
Handles agent-agent interactions.
- `checkCollisions(vector<Agent> agents)`: Detects agents in contact by identifying overlapping faces, then computes exact geodesic distances using the Vector Heat Method ([link](https://geometry-central.net/surface/algorithms/vector_heat_method/#logarithmic-map)). Creates a list of interacting agent indices.

- `fixCollision(vector<Agent>& agents)`: Implements interactions:
  - Same-type interactions (evil-evil or good-good): Elastic collisions.
  - Different-type interactions: Evil cells are phagocytosed and removed with probability `REMOVAL_PROBABILITY` (default: 70%).

**Note:** For very large meshes, running the vector heat method on sub-meshes is recommended (see for example [link](https://doi.org/10.48550/arXiv.2404.19751)).

# Field
The PDE used to implement the chemokines is the following one (as in <a href="https://doi.org/10.3389/fmicb.2015.00503">link</a>):
$$\partial_t c(x,t) = D\Delta c(x,t) - \lambda c(x,t) + S(x,t) - Q(x,t)$$
where:
- $S(x,t) = \sum_kS_0\mathbb{1}_{E_k(t)}(x)$ where $E_k(t)$ is the area covered by the k-th evil agent at time t (if alive, otherwise we remove it).
- $Q(x,t) = \sum_k\frac{k_b R_k(t)}{|G_k(t)|}c(x,t)\mathbb{1}_{G_k(t)}(x)$ where $G_k$ is the area covered by the k-th good agent at time t.

Now if we go to the FEM formulation we get to:
$$\textbf{M}\frac{d\vec{c}}{dt}  = -D\textbf{K}\vec{c} -\lambda\textbf{M}\vec{c} + \big( \int_\Omega S(x,t) \phi_j\big)_{j=1,\dots} -\big(\int_\Omega Q(x,t) \phi_j\big)_{j=1,\dots}$$
now we have to treat the sink and source term differently, in fact the sources do not change much, only if some evil agent gets killed by a good one.
On the other hand the good agents move all the time, hence using expensive integrators at every time-step would result in a bottleneck...

For this reason:
- we keep the source term as is, and we simply define: $\vec{F}:=\big( \int_\Omega S(x,t) \phi_j\big)_{j=1,\dots}$
- for the sink term first lets define $\alpha_k(t) = \frac{k_b R_k(t)}{|G_k(t)|}$ now expanding the integral we can rewrite it as:
  $$\int_\Omega Q(x,t) \phi_j = \sum_k \sum_s\alpha_k  c^s(t)\int_{G_k}\phi_s\phi_j$$

from this we can see that the integral term is just an entry of the mass matrix. We can exchange summations:
$$\sum_s(\sum_k \alpha_k  \int_{G_k}\phi_s\phi_j ) c^s(t) = (\textbf{A}\vec{c})_j$$

for an appropriate matrix $\textbf{A}=\textbf{A(t)}$ that can be constructed from the mass-matrix $\textbf{M}$.

Going back to the FEM formulation, and using Backward-Euler (with coefficient lag for $\textbf{A}$ and $\textbf{F}$) we get:
$$(\textbf{M} + \delta t (D\textbf{K}  + \lambda \textbf{M}))\vec{c}(t+\delta t) = \textbf{M}\vec{c}(t) + \delta t\vec{F}(t) - \delta t \textbf{A}(t)\vec{c}(t)$$

which we solve by using conjugate gradient method + preconditioner.

**Note**: alternatively one could also use the half-implicit scheme: $$(\textbf{M} + \delta t (D\textbf{K}  + \lambda \textbf{M}) + \delta t \textbf{A}(t))\vec{c}(t+\delta t) = \textbf{M}\vec{c}(t) + \delta t\vec{F}(t)$$

# Summary
![model](imgs/model_overview.jpg)